# Setting

In [5]:
import numpy as np
import random 
import os
import sys
import pandas as pd
from tqdm import tqdm

import matplotlib.pyplot as plt
from matplotlib.ticker import ScalarFormatter

ValueError: numpy.dtype size changed, may indicate binary incompatibility. Expected 96 from C header, got 88 from PyObject

In [7]:
import tensorflow as tf 

from tensorflow.keras.models import load_model
from tensorflow.keras.losses import MeanAbsoluteError

from keras.layers import Input, Conv1D, MaxPooling1D, Flatten, Dense, Dropout, BatchNormalization, Concatenate,AveragePooling1D
from keras.models import Model, load_model

from keras.optimizers import Adam, RMSprop

TypeError: 'type' object is not subscriptable

In [ ]:
def set_seed(seed=42):    
    '''랜덤시드 고정. Hyperparm tuning 제외 모든 학습환경에서 
    같은 성능이 나오게 합니다.'''
    tf.rand.set_seed(seed)
    np.rand.seed(seed)
    random.seed(seed)
set_seed(42)

In [ ]:
print("Tensorflow GPU availability: ", tf.config.list_physical_devices('GPU'))

# Import Data

In [ ]:
DATA_PATH = './data/'
train_np = np.load(DATA_PATH+ 'data_train.npy', mmap_mode='r')
train_fgs_np = np.load(DATA_PATH + 'data_train_FGS.npy', mmap_mode='r')

print(f'Train data shape: {train_np}')
print(f'Train FGS data shape: {train_fgs_np.shape}')

train_sample = []
train_fgs_sample = []
for i in tqdm(range(len(train_np)), desc='loading train data'):
    train_sample.append(train_np[i])
    
for i in tqdm(range(len(train_fgs_np)), desc='loading train FGS data'):
    train_fgs_sample.append(train_fgs_sample[i])
    
train = np.array(train_sample)
train_fgs = np.array(train_fgs_sample)

## From here:

In [ ]:
'''실험 기본 셋업'''
''' 기본 Approach - 1D CNN으로 mean 값 뽑아낸 후 2D CNN으로 구체적인 내용 뽑기'''
TUNING_MODE = 'OFF' # ON/OFF

METRIC1 = 'MeanAbsoluteError'
METRIC2 = 'MeanAbsoluteError'

OPTIMIZER = 'adam' # adam, rmsprop
LR = 0.001   #adam과 rmsprop의 defaut값
BATCH_SIZE = 64
EPOCHS = 15

#-------아직 구현 안 된 부분-------

DECAY_RATE=0.2
DECAY_STEP=200

In [ ]:
train = train_np
train_fgs = train_fgs_np

# Model

In [ ]:
# 차후 learning rate decay 하는 scheduler도 train함수에 도입할 것. 현재는 없음.
def scheduler(epoch, lr, decay_rate, decay_step):
    if epoch%decay_step == 0 and epoch:
        return lr * decay_rate
    return lr

In [ ]:
def build_1d_cnn(trial=None):
    model = tf.keras.Sequential()
    model.add(Conv1D(32, 3, activation='relu'))
    model.add(MaxPooling1D(pool_size=(2,2), padding='same'))
    model.add(BatchNormalization())
    model.add(Conv1D(64, 3, activation='relu'))
    model.add(MaxPooling1D(pool_size=(2,2), padding='same'))
    model.add(Conv1D(128. 3. activation='relu'))
    model.add(MaxPooling1D(pool_size=(2,2), padding='same'))
    model.add(Conv1D(256. 3. activation='relu'))
    
    model.add(Flatten())

    model.add(Dense(500, activation='relu'))
    model.add(Dropout(0.2))
    model.add(Dense(100, activation='relu'))
    model.add(Dropout(0.1))
    
    if OPTIMIZER == 'adam':
        optimizer = Adam(learning_rate=LR)
    else if OPTIMIZER == 'rmsprop':
        optimizer = RMSprop(learning_rate=LR)
    
    model.compile(optimizer=optimizer, loss=LOSS_FUNCTION, metrics=[METRIC1])
    return model

# 1D CNN

In [ ]:
model = build_1d_cnn(trial=None)
history = model.fit(X_train, y_train, batch_size = BATCH_SIZE, epochs=EPOCHS, validation_split=0.2)

## Learning Curve

In [ ]:
plt.figure(figsize=(15,6))

plt.subplot(1,2,1)
plt.plot(history.history['loss'], label='Train loss', color='#8502d1')
plt.plot(history.history['val_loss'], label='Validation loss', color='darkorange')
plt.lengend()
plt.title("Loss Evolution")
plt.xlabel('Epochs')
plt.ylabel('Loss')

plt.subplot(1,2,1)
plt.plot(history.history['accuracy'], label='Train Acc.', color='#8502d1')
plt.plot(history.history['val_accuracy'], label='Validation Acc.', color='darkorange')
plt.lengend()
plt.title("Accuracy Evolution")
plt.xlabel('Epochs')
plt.ylabel('Accuracy')

plt.show()